In [1]:
import os
import sys
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from scipy.io import loadmat

In [2]:
from pyod.models.pca import PCA
from pyod.models.mcd import MCD
from pyod.models.ocsvm import OCSVM
from pyod.models.lof import LOF
from pyod.models.cblof import CBLOF
from pyod.models.knn import KNN
from pyod.models.hbos import HBOS
from pyod.models.abod import ABOD
from pyod.models.iforest import IForest
from pyod.models.feature_bagging import FeatureBagging

In [3]:
from pyod.utils.utility import standardizer
from pyod.utils.utility import precision_n_scores
from sklearn.metrics import roc_auc_score

In [4]:
mat_file_list = ['arrhythmia.mat',
                 'cardio.mat',
                 'glass.mat',
                 'ionosphere.mat',
                 'letter.mat',
                 'lympho.mat',
                 'mnist.mat',
                 'musk.mat',
                 'optdigits.mat',
                 'pendigits.mat',
                 'pima.mat',
                 'satellite.mat',
                 'satimage-2.mat',
                 'shuttle.mat',
                 'vertebral.mat',
                 'vowels.mat',
                 'wbc.mat']
mat_file_list

['arrhythmia.mat',
 'cardio.mat',
 'glass.mat',
 'ionosphere.mat',
 'letter.mat',
 'lympho.mat',
 'mnist.mat',
 'musk.mat',
 'optdigits.mat',
 'pendigits.mat',
 'pima.mat',
 'satellite.mat',
 'satimage-2.mat',
 'shuttle.mat',
 'vertebral.mat',
 'vowels.mat',
 'wbc.mat']

In [5]:
from scipy.io import loadmat

In [6]:
data=loadmat('C:\\Users\\Dhruvraj\\Desktop\\LETS UPGRADE\\LetsUpgrade AI ML\\csv\\anomaly\\cardio.mat')

In [7]:
data

{'__header__': b'MATLAB 5.0 MAT-file, written by Octave 3.8.0, 2014-12-18 10:48:09 UTC',
 '__version__': '1.0',
 '__globals__': [],
 'X': array([[ 0.00491231,  0.69319077, -0.20364049, ...,  0.23149795,
         -0.28978574, -0.49329397],
        [ 0.11072935, -0.07990259, -0.20364049, ...,  0.09356344,
         -0.25638541, -0.49329397],
        [ 0.21654639, -0.27244466, -0.20364049, ...,  0.02459619,
         -0.25638541,  1.14001753],
        ...,
        [-0.41835583, -0.91998844, -0.16463485, ..., -1.49268341,
          0.24461959, -0.49329397],
        [-0.41835583, -0.91998844, -0.15093411, ..., -1.42371616,
          0.14441859, -0.49329397],
        [-0.41835583, -0.91998844, -0.20364049, ..., -1.28578165,
          3.58465295, -0.49329397]]),
 'y': array([[0.],
        [0.],
        [0.],
        ...,
        [1.],
        [1.],
        [1.]])}

In [8]:
data.keys()

dict_keys(['__header__', '__version__', '__globals__', 'X', 'y'])

In [9]:

data.values()

dict_values([b'MATLAB 5.0 MAT-file, written by Octave 3.8.0, 2014-12-18 10:48:09 UTC', '1.0', [], array([[ 0.00491231,  0.69319077, -0.20364049, ...,  0.23149795,
        -0.28978574, -0.49329397],
       [ 0.11072935, -0.07990259, -0.20364049, ...,  0.09356344,
        -0.25638541, -0.49329397],
       [ 0.21654639, -0.27244466, -0.20364049, ...,  0.02459619,
        -0.25638541,  1.14001753],
       ...,
       [-0.41835583, -0.91998844, -0.16463485, ..., -1.49268341,
         0.24461959, -0.49329397],
       [-0.41835583, -0.91998844, -0.15093411, ..., -1.42371616,
         0.14441859, -0.49329397],
       [-0.41835583, -0.91998844, -0.20364049, ..., -1.28578165,
         3.58465295, -0.49329397]]), array([[0.],
       [0.],
       [0.],
       ...,
       [1.],
       [1.],
       [1.]])])

In [10]:
type(data['X']),data['X'].shape

(numpy.ndarray, (1831, 21))

In [11]:
type(data['y']),data['y'].shape

(numpy.ndarray, (1831, 1))

In [12]:
df_columns = ['Data', '#Samples', '# Dimensions', 'Outlier Perc',
              'ABOD', 'CBLOF', 'FB', 'HBOS', 'IForest', 'KNN', 'LOF', 'MCD',
              'OCSVM', 'PCA']

In [13]:
roc_df = pd.DataFrame(columns=df_columns)
roc_df

,Data,#Samples,# Dimensions,Outlier Perc,ABOD,CBLOF,FB,HBOS,IForest,KNN,LOF,MCD,OCSVM,PCA


In [14]:
prn_df = pd.DataFrame(columns=df_columns)
prn_df


,Data,#Samples,# Dimensions,Outlier Perc,ABOD,CBLOF,FB,HBOS,IForest,KNN,LOF,MCD,OCSVM,PCA


In [15]:
time_df = pd.DataFrame(columns=df_columns)
time_df

,Data,#Samples,# Dimensions,Outlier Perc,ABOD,CBLOF,FB,HBOS,IForest,KNN,LOF,MCD,OCSVM,PCA


In [17]:
from time import time
random_state = np.random.RandomState(30)

for mat_file in mat_file_list:
    print("\n... Processing", mat_file, '...')
    mat = loadmat('C:\\Users\\Dhruvraj\\Desktop\\LETS UPGRADE\\LetsUpgrade AI ML\\csv\\anomaly\\'+mat_file)

    X = mat['X']
    y = mat['y'].ravel()
    outliers_fraction = np.count_nonzero(y) / len(y)
    outliers_percentage = round(outliers_fraction * 100, ndigits=4)

    # construct containers for saving results
    roc_list = [mat_file[:-4], X.shape[0], X.shape[1], outliers_percentage]
    prn_list = [mat_file[:-4], X.shape[0], X.shape[1], outliers_percentage]
    time_list = [mat_file[:-4], X.shape[0], X.shape[1], outliers_percentage]

    # 60% data for training and 40% for testing
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4,
                                                        random_state=random_state)

    # standardizing data for processing
    X_train_norm, X_test_norm = standardizer(X_train, X_test)

    classifiers = {'Angle-based Outlier Detector (ABOD)': ABOD(
        contamination=outliers_fraction),
        'Cluster-based Local Outlier Factor': CBLOF(
            contamination=outliers_fraction, check_estimator=False,
            random_state=random_state),
        'Feature Bagging': FeatureBagging(contamination=outliers_fraction,
                                          random_state=random_state),
        'Histogram-base Outlier Detection (HBOS)': HBOS(
            contamination=outliers_fraction),
        'Isolation Forest': IForest(contamination=outliers_fraction,
                                    random_state=random_state),
        'K Nearest Neighbors (KNN)': KNN(contamination=outliers_fraction),
        'Local Outlier Factor (LOF)': LOF(
            contamination=outliers_fraction),
        'Minimum Covariance Determinant (MCD)': MCD(
            contamination=outliers_fraction, random_state=random_state),
        'One-class SVM (OCSVM)': OCSVM(contamination=outliers_fraction),
        'Principal Component Analysis (PCA)': PCA(
            contamination=outliers_fraction, random_state=random_state),
    }

    for clf_name, clf in classifiers.items():
        t0 = time()
        clf.fit(X_train_norm)
        test_scores = clf.decision_function(X_test_norm)
        t1 = time()
        duration = round(t1 - t0, ndigits=4)
        time_list.append(duration)

        roc = round(roc_auc_score(y_test, test_scores), ndigits=4)
        prn = round(precision_n_scores(y_test, test_scores), ndigits=4)

        print('{clf_name} ROC:{roc}, precision @ rank n:{prn}, '
              'execution time: {duration}s'.format(
            clf_name=clf_name, roc=roc, prn=prn, duration=duration))

        roc_list.append(roc)
        prn_list.append(prn)

    temp_df = pd.DataFrame(time_list).transpose()
    temp_df.columns = df_columns
    time_df = pd.concat([time_df, temp_df], axis=0)

    temp_df = pd.DataFrame(roc_list).transpose()
    temp_df.columns = df_columns
    roc_df = pd.concat([roc_df, temp_df], axis=0)

    temp_df = pd.DataFrame(prn_list).transpose()
    temp_df.columns = df_columns
    prn_df = pd.concat([prn_df, temp_df], axis=0)


... Processing arrhythmia.mat ...
Angle-based Outlier Detector (ABOD) ROC:0.7895, precision @ rank n:0.3333, execution time: 0.3473s


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:973: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 0.25.
  warnings.warn("'n_jobs' was deprecated in version 0.23 and will be"


Cluster-based Local Outlier Factor ROC:0.7447, precision @ rank n:0.4167, execution time: 0.2942s
Feature Bagging ROC:0.7688, precision @ rank n:0.375, execution time: 1.299s
Histogram-base Outlier Detection (HBOS) ROC:0.801, precision @ rank n:0.4583, execution time: 0.1582s
Isolation Forest ROC:0.7853, precision @ rank n:0.5, execution time: 1.087s
K Nearest Neighbors (KNN) ROC:0.7625, precision @ rank n:0.4167, execution time: 0.3276s
Local Outlier Factor (LOF) ROC:0.776, precision @ rank n:0.375, execution time: 0.1588s


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:647: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn("The covariance matrix associated to your dataset "


Minimum Covariance Determinant (MCD) ROC:0.7678, precision @ rank n:0.375, execution time: 9.0441s
One-class SVM (OCSVM) ROC:0.7649, precision @ rank n:0.4167, execution time: 0.1301s
Principal Component Analysis (PCA) ROC:0.7622, precision @ rank n:0.4167, execution time: 0.2907s

... Processing cardio.mat ...
Angle-based Outlier Detector (ABOD) ROC:0.6001, precision @ rank n:0.1746, execution time: 0.9424s


...

Angle-based Outlier Detector (ABOD) ROC:0.9209, precision @ rank n:0.4375, execution time: 0.3507s
Cluster-based Local Outlier Factor ROC:0.9396, precision @ rank n:0.4118, execution time: 0.1335s


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:973: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 0.25.
  warnings.warn("'n_jobs' was deprecated in version 0.23 and will be"


Feature Bagging ROC:0.9415, precision @ rank n:0.3529, execution time: 0.3769s
Histogram-base Outlier Detection (HBOS) ROC:0.7061, precision @ rank n:0.1176, execution time: 0.0051s
Isolation Forest ROC:0.7372, precision @ rank n:0.1765, execution time: 0.7028s
K Nearest Neighbors (KNN) ROC:0.9672, precision @ rank n:0.5294, execution time: 0.0926s
Local Outlier Factor (LOF) ROC:0.9374, precision @ rank n:0.2941, execution time: 0.0374s
Minimum Covariance Determinant (MCD) ROC:0.7191, precision @ rank n:0.0588, execution time: 2.3377s
One-class SVM (OCSVM) ROC:0.7922, precision @ rank n:0.2941, execution time: 0.0387s
Principal Component Analysis (PCA) ROC:0.5946, precision @ rank n:0.1176, execution time: 0.0037s

... Processing wbc.mat ...
Angle-based Outlier Detector (ABOD) ROC:0.9005, precision @ rank n:0.3333, execution time: 0.1811s
Cluster-based Local Outlier Factor ROC:0.9075, precision @ rank n:0.4444, execution time: 0.1022s
Feature Bagging ROC:0.9192, precision @ rank n:0.44

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:973: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 0.25.
  warnings.warn("'n_jobs' was deprecated in version 0.23 and will be"


Histogram-base Outlier Detection (HBOS) ROC:0.9503, precision @ rank n:0.6667, execution time: 0.011s
Isolation Forest ROC:0.9316, precision @ rank n:0.4444, execution time: 0.4551s
K Nearest Neighbors (KNN) ROC:0.9277, precision @ rank n:0.4444, execution time: 0.0213s
Local Outlier Factor (LOF) ROC:0.9239, precision @ rank n:0.5556, execution time: 0.008s
Minimum Covariance Determinant (MCD) ROC:0.9021, precision @ rank n:0.3333, execution time: 0.2224s
One-class SVM (OCSVM) ROC:0.9277, precision @ rank n:0.4444, execution time: 0.001s
Principal Component Analysis (PCA) ROC:0.9231, precision @ rank n:0.4444, execution time: 0.0014s


In [18]:
roc_df

,Data,#Samples,# Dimensions,Outlier Perc,ABOD,CBLOF,FB,HBOS,IForest,KNN,LOF,MCD,OCSVM,PCA
0,arrhythmia,452,274,14.6018,0.7456,0.7901,0.7714,0.8159,0.8458,0.783,0.774,0.7882,0.7811,0.7811
0,cardio,1831,21,9.6122,0.6097,0.7981,0.5617,0.8472,0.9226,0.7219,0.5534,0.8604,0.9263,0.9463
0,glass,214,9,4.2056,0.7975,0.8988,0.8469,0.7309,0.6247,0.8691,0.8642,0.7877,0.4247,0.4889
0,ionosphere,351,33,35.8974,0.9355,0.89,0.8849,0.5674,0.8625,0.9129,0.8846,0.9536,0.8163,0.761
0,letter,1600,32,6.25,0.8728,0.7704,0.8493,0.6206,0.6468,0.8559,0.8401,0.7924,0.6219,0.5607
0,lympho,148,18,4.0541,0.8973,0.9241,0.9643,0.9955,0.9777,0.9598,0.9643,0.9375,0.9643,0.9732
0,mnist,7603,100,9.2069,0.7859,0.8365,0.7414,0.5761,0.8178,0.8527,0.743,0.8886,0.8418,0.8416
0,musk,3062,166,3.1679,0.2095,1,0.6355,0.9998,0.996,0.7444,0.6699,0.999,1,1
0,optdigits,5216,64,2.8758,0.5233,0.7906,0.4683,0.8565,0.7721,0.3835,0.4596,0.389,0.5178,0.5225
0,pendigits,6870,16,2.2707,0.6131,0.9707,0.3381,0.9171,0.9288,0.7533,0.3332,0.8312,0.9329,0.9433


In [19]:
prn_df

,Data,#Samples,# Dimensions,Outlier Perc,ABOD,CBLOF,FB,HBOS,IForest,KNN,LOF,MCD,OCSVM,PCA
0,arrhythmia,452,274,14.6018,0.4516,0.4516,0.4194,0.4516,0.5806,0.4516,0.4194,0.4194,0.4516,0.4516
0,cardio,1831,21,9.6122,0.2576,0.4697,0.1212,0.3788,0.4545,0.3182,0.0909,0.4394,0.4545,0.5758
0,glass,214,9,4.2056,0,0,0,0,0,0,0,0,0,0
0,ionosphere,351,33,35.8974,0.8333,0.7917,0.7292,0.2917,0.6875,0.8333,0.7083,0.875,0.7083,0.5833
0,letter,1600,32,6.25,0.3158,0.1579,0.3158,0.0789,0.1316,0.2368,0.2895,0.1579,0.1316,0.0789
0,lympho,148,18,4.0541,0.5,0.5,0.5,0.75,0.5,0.5,0.5,0.5,0.5,0.5
0,mnist,7603,100,9.2069,0.3962,0.4075,0.3396,0.1057,0.3132,0.4151,0.3245,0.4415,0.3547,0.3472
0,musk,3062,166,3.1679,0.125,1,0.3125,0.9688,0.8125,0.375,0.3125,0.9375,1,1
0,optdigits,5216,64,2.8758,0.0455,0,0.0758,0.197,0.0455,0,0.0758,0,0,0
0,pendigits,6870,16,2.2707,0.0469,0.3906,0.0156,0.3281,0.3438,0.0469,0.0312,0.0938,0.3594,0.3125


In [20]:
time_df

,Data,#Samples,# Dimensions,Outlier Perc,ABOD,CBLOF,FB,HBOS,IForest,KNN,LOF,MCD,OCSVM,PCA
0,arrhythmia,452,274,14.6018,2.8495,3.1863,1.4847,3.7958,0.8897,0.2006,0.1657,6.8453,0.09,0.2944
0,cardio,1831,21,9.6122,0.8855,0.2418,1.5842,0.0114,0.825,0.3199,0.2752,3.4449,0.1723,0.0239
0,glass,214,9,4.2056,0.077,0.1017,0.062,0.017,0.7663,0.0312,0,0.0871,0,0
0,ionosphere,351,33,35.8974,0.1399,0.0996,0.139,0.0174,0.7288,0.0324,0.0081,0.54,0.0169,0
0,letter,1600,32,6.25,0.8709,0.2352,1.6283,0.0204,0.8817,0.2912,0.1858,10.9702,0.1612,0.0178
0,lympho,148,18,4.0541,0.0611,0.1028,0.071,0.0169,0.6629,0.0119,0.0086,0.1533,0,0.0043
0,mnist,7603,100,9.2069,15.621,1.494,94.089,0.116,4.1425,14.787,13.8581,31.5735,9.747,0.327
0,musk,3062,166,3.1679,4.8472,0.4844,24.2625,0.1137,2.413,3.7544,3.6666,134.129,2.5026,0.4518
0,optdigits,5216,64,2.8758,5.6729,0.5402,27.2354,0.0752,2.1165,3.7783,3.433,14.143,2.8119,0.116
0,pendigits,6870,16,2.2707,3.6925,0.4209,10.9487,0.0257,2.0733,1.8122,1.4726,6.5873,1.9457,0.0104
